In [78]:
import pandas as pd

In [292]:
auto_df = pd.read_csv("automatability.csv") #to transpose
relative_emp_df = pd.read_csv("relativeEmployment.csv") #to transpose
similar_df = pd.read_csv("newsimilarity.csv")
wagechange_df = pd.read_csv("wageChange.csv")

# all_csvs = [auto_df, relative_emp_df, similar_df, wagechange_df]
# for csv in all_csvs:
# auto_df.columns.values[0]='Job_Compared'
# relative_emp_df.columns.values[0]='Job_Compared'
# similar_df.columns.values[0]='Job_Compared'
# wagechange_df.columns.values[0]='Job_Compared'
            

In [293]:
auto_df = auto_df.transpose()
auto_df.columns = auto_df.iloc[0]
auto_df.drop(auto_df.index[0], inplace=True)

relative_emp_df = relative_emp_df.transpose()
relative_emp_df.columns = relative_emp_df.iloc[0]
relative_emp_df.drop(relative_emp_df.index[0], inplace=True)



# # auto_df=auto_df.set_index('Job Compared')
# # relative_emp_df=relative_emp_df.set_index('Job Compared')
# # similar_df=similar_df.set_index('Job Compared')
# # wagechange_df = wagechange_df.set_index('Job Compared')

In [295]:
wagechange_df

,Job_Compared,Recreational Therapists,"First-Line Supervisors of Mechanics, Installers, and Repairers",Emergency Management Directors,Mental Health and Substance Abuse Social Workers,Audiologists,Occupational Therapists,Orthotists and Prosthetists,Healthcare Social Workers,Oral and Maxillofacial Surgeons,...,Library Technicians,New Accounts Clerks,Photographic Process Workers and Processing Machine Operators,Tax Preparers,Cargo and Freight Agents,Watch Repairers,Insurance Underwriters,Mathematical Technicians,"Sewers, Hand",Telemarketers
0,Recreational Therapists,0.000000,0.384727,0.619838,-0.006433,0.645362,0.737497,0.450923,0.151899,3.832330,...,-0.278274,-0.242996,-0.341357,-0.059141,-0.081760,-0.175763,0.566300,0.213737,-0.468147,-0.436190
1,"First-Line Supervisors of Mechanics, Installer...",-0.277836,0.000000,0.169789,-0.282482,0.188221,0.254758,0.047805,-0.168140,2.489735,...,-0.478795,-0.453319,-0.524352,-0.320545,-0.336880,-0.404765,0.131125,-0.123483,-0.615915,-0.592837
2,Emergency Management Directors,-0.382654,-0.145145,0.000000,-0.386626,0.015757,0.072636,-0.104279,-0.288880,1.983218,...,-0.554445,-0.532667,-0.593390,-0.419165,-0.433128,-0.491161,-0.033051,-0.250705,-0.671663,-0.651934
3,Mental Health and Substance Abuse Social Workers,0.006475,0.393693,0.630326,0.000000,0.656015,0.748747,0.460317,0.159357,3.863617,...,-0.273601,-0.238095,-0.337093,-0.053049,-0.075815,-0.170426,0.576441,0.221596,-0.464703,-0.432540
4,Audiologists,-0.392231,-0.158406,-0.015513,-0.396141,0.000000,0.055997,-0.118174,-0.299912,1.936940,...,-0.561357,-0.539917,-0.599697,-0.428175,-0.441922,-0.499054,-0.048051,-0.262328,-0.676756,-0.657334
5,Occupational Therapists,-0.424460,-0.203034,-0.067718,-0.428162,-0.053028,0.000000,-0.164935,-0.337036,1.781201,...,-0.584617,-0.564314,-0.620924,-0.458498,-0.471516,-0.525618,-0.098531,-0.301445,-0.693897,-0.675505
6,Orthotists and Prosthetists,-0.310784,-0.045624,0.116419,-0.315217,0.134010,0.197511,0.000000,-0.206093,2.330521,...,-0.502574,-0.478261,-0.546053,-0.351545,-0.367134,-0.431922,0.079519,-0.163473,-0.633438,-0.611413
7,Healthcare Social Workers,-0.131868,0.202126,0.406233,-0.137453,0.428391,0.508377,0.259593,0.000000,3.195100,...,-0.373446,-0.342821,-0.428211,-0.183210,-0.202846,-0.284453,0.359755,0.053684,-0.538281,-0.510539
8,Oral and Maxillofacial Surgeons,-0.793061,-0.713445,-0.664792,-0.794392,-0.659510,-0.640443,-0.699747,-0.761627,0.000000,...,-0.850646,-0.843346,-0.863701,-0.805299,-0.809980,-0.829433,-0.675871,-0.748830,-0.889939,-0.883325
9,First-Line Supervisors of Fire Fighting and Pr...,-0.374562,-0.133939,0.013108,-0.378585,0.029072,0.086697,-0.092537,-0.279559,2.022323,...,-0.548605,-0.526541,-0.588060,-0.411551,-0.425698,-0.484491,-0.020376,-0.240883,-0.667359,-0.647372


In [303]:
# relative_emp_df = relative_emp_df.reset_index()

# relative_emp_df.index.names = ['sdf']

relative_emp_df
# df=df.rename(columns = {'two':'new_name'})


Job_Compared,index,Recreational Therapists,"First-Line Supervisors of Mechanics, Installers, and Repairers",Emergency Management Directors,Mental Health and Substance Abuse Social Workers,Audiologists,Occupational Therapists,Orthotists and Prosthetists,Healthcare Social Workers,Oral and Maxillofacial Surgeons,...,Library Technicians,New Accounts Clerks,Photographic Process Workers and Processing Machine Operators,Tax Preparers,Cargo and Freight Agents,Watch Repairers,Insurance Underwriters,Mathematical Technicians,"Sewers, Hand",Telemarketers
sdf,,,,,,,,,,,,,,,,,,,,,
0,Recreational Therapists,1,0.1,1,0.158716,1,0.153299,1,0.113615,1,...,0.193769,0.434783,0.684828,0.258461,0.203554,1,0.19749,1,1,0.1
1,"First-Line Supervisors of Mechanics, Installer...",1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,Emergency Management Directors,0.528729,0.1,1,0.1,0.777417,0.1,1,0.1,1,...,0.102452,0.229882,0.362089,0.136656,0.107625,1,0.104419,1,1,0.1
3,Mental Health and Substance Abuse Social Workers,1,0.251561,1,1,1,0.965868,1,0.715837,1,...,1,1,1,1,1,1,1,1,1,0.529704
4,Audiologists,0.68011,0.1,1,0.107945,1,0.10426,1,0.1,1,...,0.131785,0.2957,0.465759,0.175782,0.138439,1,0.134315,1,1,0.1
5,Occupational Therapists,1,0.26045,1,1,1,1,1,0.741134,1,...,1,1,1,1,1,1,1,1,1,0.548423
6,Orthotists and Prosthetists,0.414365,0.1,0.783699,0.1,0.609261,0.1,1,0.1,1,...,0.1,0.180159,0.283768,0.107097,0.1,1,0.1,1,1,0.1
7,Healthcare Social Workers,1,0.351422,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0.739979
8,Oral and Maxillofacial Surgeons,0.297238,0.1,0.562173,0.1,0.437043,0.1,0.717333,0.1,1,...,0.1,0.129234,0.203557,0.1,0.1,1,0.1,1,0.82263,0.1


In [215]:
auto_df = auto_df[['Job_Compared','Heavy and Tractor-Trailer Truck Drivers','Athletes and Sports Competitors','Compensation and Benefits Managers','Construction Laborers']]
relative_emp_df = relative_emp_df[['Job_Compared','Heavy and Tractor-Trailer Truck Drivers','Athletes and Sports Competitors','Compensation and Benefits Managers','Construction Laborers']]
similar_df = similar_df[['Job_Compared','Heavy and Tractor-Trailer Truck Drivers','Athletes and Sports Competitors','Compensation and Benefits Managers','Construction Laborers']]
wagechange_df = wagechange_df[['Job_Compared','Heavy and Tractor-Trailer Truck Drivers','Athletes and Sports Competitors','Compensation and Benefits Managers','Construction Laborers']]

In [218]:
auto_df=auto_df.set_index('Job_Compared')
relative_emp_df=relative_emp_df.set_index('Job_Compared')
similar_df=similar_df.set_index('Job_Compared')
wagechange_df = wagechange_df.set_index('Job_Compared')

,Heavy and Tractor-Trailer Truck Drivers,Athletes and Sports Competitors,Compensation and Benefits Managers,Construction Laborers
Job_Compared,,,,
Recreational Therapists,-0.095455,0.737497,1.633326,-0.213737
"First-Line Supervisors of Mechanics, Installers, and Repairers",-0.346771,0.254758,0.901693,-0.432189
Emergency Management Directors,-0.441583,0.072636,0.625673,-0.514604
Mental Health and Substance Abuse Social Workers,-0.089599,0.748747,1.650376,-0.208647
Audiologists,-0.450246,0.055997,0.600454,-0.522134


In [221]:
auto_df = auto_df.add_prefix('auto_')
relative_emp_df = relative_emp_df.add_prefix('empl_')
similar_df = similar_df.add_prefix('similar_')
wagechange_df = wagechange_df.add_prefix('wage_')


In [223]:
auto_df= auto_df= auto_df.rename(columns=lambda x: x.replace(' ','_'))
relative_emp_df= relative_emp_df.rename(columns=lambda x: x.replace(' ','_'))
similar_df= similar_df.rename(columns=lambda x: x.replace(' ','_'))
wagechange_df= wagechange_df.rename(columns=lambda x: x.replace(' ','_'))

In [138]:
auto_df= auto_df= auto_df.rename(columns=lambda x: x.replace(' ','_'))
relative_emp_df= relative_emp_df.rename(columns=lambda x: x.replace(' ','_'))
similar_df= similar_df.rename(columns=lambda x: x.replace(' ','_'))
wagechange_df= wagechange_df.rename(columns=lambda x: x.replace(' ','_'))

In [234]:
truckers_wage = wagechange_df[wagechange_df.columns[0]].reset_index()
athletes_wage = wagechange_df[wagechange_df.columns[1]].reset_index()
comp_managers_wage = wagechange_df[wagechange_df.columns[2]].reset_index()
laborers_wage = wagechange_df[wagechange_df.columns[3]].reset_index()

    
truckers_similarity = similar_df[similar_df.columns[0]].reset_index()
athletes_similarity = similar_df[similar_df.columns[1]].reset_index()
comp_managers_similarity = similar_df[similar_df.columns[2]].reset_index()
laborers_similarity = similar_df[similar_df.columns[3]].reset_index()


truckers_relative_emp = relative_emp_df[relative_emp_df.columns[0]].reset_index()
athletes_relative_emp = relative_emp_df[relative_emp_df.columns[1]].reset_index()
comp_managers_relative_emp = relative_emp_df[relative_emp_df.columns[2]].reset_index()
laborers_relative_emp = relative_emp_df[relative_emp_df.columns[3]].reset_index()


truckers_automatability = auto_df[auto_df.columns[0]].reset_index()
athletes_automatability = auto_df[auto_df.columns[1]].reset_index()
comp_managers_automatability = auto_df[auto_df.columns[2]].reset_index()
laborers_automatability = auto_df[auto_df.columns[3]].reset_index()

In [248]:
dfs_trucker = [truckers_wage, truckers_similarity, truckers_relative_emp, truckers_automatability]

dfs_athlete = [athletes_wage,athletes_similarity,athletes_relative_emp,athletes_automatability]

dfs_comp_managers = [comp_managers_wage,comp_managers_similarity,comp_managers_relative_emp,comp_managers_automatability]

dfs_laborers = [laborers_wage,laborers_similarity,laborers_relative_emp,laborers_automatability]


df_trucker_final      =reduce(lambda left,right: pd.merge(left,right,on='Job_Compared'), dfs_trucker)
df_athlete_final      =reduce(lambda left,right: pd.merge(left,right,on='Job_Compared'), dfs_athlete)
df_comp_managers_final=reduce(lambda left,right: pd.merge(left,right,on='Job_Compared'), dfs_comp_managers)
df_laborers_final     =reduce(lambda left,right: pd.merge(left,right,on='Job_Compared'), dfs_laborers)


In [251]:
df_trucker_final = df_trucker_final.reset_index()
df_athlete_final = df_athlete_final.reset_index("Job_Compared")
df_comp_managers_final = df_comp_managers_final.reset_index("Job_Compared")
df_laborers_final = df_laborers_final.reset_index("Job_Compared")


In [253]:
raw_wage_auto_jobs = pd.read_csv("jobs.csv")
raw_wage_auto_jobs= raw_wage_auto_jobs
# .set_index("Occupation")


In [255]:
laborers = pd.merge(raw_wage_auto_jobs,df_laborers_final, right_on='Job_Compared',left_on='Occupation')
truckers = pd.merge(raw_wage_auto_jobs,df_trucker_final, right_on='Job_Compared',left_on='Occupation')
athletes = pd.merge(raw_wage_auto_jobs,df_athlete_final, right_on='Job_Compared',left_on='Occupation')
comp_managers = pd.merge(raw_wage_auto_jobs,df_comp_managers_final, right_on='Job_Compared',left_on='Occupation')

In [257]:
laborers.to_csv("laborers.csv")
truckers.to_csv("truckers.csv")
athletes.to_csv("athletes.csv")
comp_managers.to_csv("comp_managers.csv")

In [77]:
# auto_df.to_csv("auto.csv")
# relative_emp.to_csv("relative_emp.csv")
# similar_df.to_csv("simlar.csv")
# wagechange.to_csv("wage.csv")